In [ ]:
# contacts.py
import argparse, json, os, sys
from datetime import datetime

APP_DIR = os.path.join(os.path.expanduser("~"), ".contacts_cli")
DB_PATH = os.path.join(APP_DIR, "contacts.json")

def ensure_storage():
    os.makedirs(APP_DIR, exist_ok=True)
    if not os.path.exists(DB_PATH):
        with open(DB_PATH, "w", encoding="utf-8") as f:
            json.dump([], f, ensure_ascii=False, indent=2)

def load_contacts():
    ensure_storage()
    with open(DB_PATH, "r", encoding="utf-8") as f:
        return json.load(f)

def save_contacts(items):
    with open(DB_PATH, "w", encoding="utf-8") as f:
        json.dump(items, f, ensure_ascii=False, indent=2)

def next_id(items):
    return (items[-1]["id"] + 1) if items else 1

# ---------- دستورات ----------
def cmd_add(args):
    items = load_contacts()
    c = {
        "id": next_id(items),
        "name": args.name.strip(),
        "phone": args.phone.strip() if args.phone else "",
        "email": args.email.strip() if args.email else "",
        "created_at": datetime.now().strftime("%Y-%m-%d %H:%M"),
    }
    if not c["name"]:
        print("❗ نام نمی‌تواند خالی باشد.", file=sys.stderr); sys.exit(1)
    items.append(c)
    save_contacts(items)
    print(f"✔ اضافه شد: #{c['id']} | {c['name']} | {c['phone']} {c['email']}")

def cmd_list(args):
    items = load_contacts()
    q = (args.search or "").strip().lower()
    if q:
        items = [c for c in items if q in c["name"].lower() or q in c["phone"] or q in c["email"].lower()]
    items = sorted(items, key=lambda x: x["name"])
    if not items:
        print("هیچ مخاطبی پیدا نشد."); return
    print("ID  نام                      تلفن            ایمیل")
    print("--  -----------------------  --------------- --------------------------")
    for c in items:
        print(f"{str(c['id']).ljust(2)}  {c['name'][:23].ljust(23)}  {c['phone'][:15].ljust(15)} {c['email']}")
    print(f"Total: {len(items)}")

def cmd_delete(args):
    items = load_contacts()
    before = len(items)
    items = [c for c in items if c["id"] != args.id]
    if len(items) == before:
        print("❗ چنین ID ای پیدا نشد.", file=sys.stderr); sys.exit(1)
    save_contacts(items)
    print(f"🗑 حذف شد: #{args.id}")

def cmd_show(args):
    items = load_contacts()
    for c in items:
        if c["id"] == args.id:
            print(json.dumps(c, ensure_ascii=False, indent=2)); return
    print("❗ چنین ID ای پیدا نشد.", file=sys.stderr); sys.exit(1)

def cmd_update(args):
    items = load_contacts()
    for c in items:
        if c["id"] == args.id:
            if args.name is not None:  c["name"] = args.name.strip()
            if args.phone is not None: c["phone"] = args.phone.strip()
            if args.email is not None: c["email"] = args.email.strip()
            save_contacts(items)
            print(f"✏️ به‌روزرسانی شد: #{args.id}"); return
    print("❗ چنین ID ای پیدا نشد.", file=sys.stderr); sys.exit(1)

# ---------- Parser ----------
def build_parser():
    p = argparse.ArgumentParser(prog="contacts", description="دفترچه مخاطبین ساده (JSON)")
    sub = p.add_subparsers(dest="cmd", required=True)

    p_add = sub.add_parser("add", help="افزودن مخاطب")
    p_add.add_argument("name", help="نام و نام‌خانوادگی")
    p_add.add_argument("--phone", help="شماره تلفن")
    p_add.add_argument("--email", help="ایمیل")
    p_add.set_defaults(func=cmd_add)

    p_list = sub.add_parser("list", help="نمایش لیست مخاطبین")
    p_list.add_argument("--search", help="جست‌وجو در نام/تلفن/ایمیل")
    p_list.set_defaults(func=cmd_list)

    p_del = sub.add_parser("delete", help="حذف مخاطب با ID")
    p_del.add_argument("id", type=int)
    p_del.set_defaults(func=cmd_delete)

    p_show = sub.add_parser("show", help="نمایش جزئیات یک مخاطب")
    p_show.add_argument("id", type=int)
    p_show.set_defaults(func=cmd_show)

    p_upd = sub.add_parser("update", help="ویرایش فیلدهای مخاطب")
    p_upd.add_argument("id", type=int)
    p_upd.add_argument("--name")
    p_upd.add_argument("--phone")
    p_upd.add_argument("--email")
    p_upd.set_defaults(func=cmd_update)

    return p

def main():
    parser = build_parser()
    args = parser.parse_args()
    args.func(args)

if __name__ == "__main__":
    main()
